In [1]:
import requests
import xml.etree.ElementTree
import time
import os
import csv

import pandas

In [2]:
def get_rxnorm_ingredients(rxcui):
    """
    Return a generator of RxNorm ingredients related to the provided RxNorm concept.
    Each ingredient is a (rxcui, name, umlscui) tuple. This function calls the
    RxNormAPI as documented here:
    http://rxnav.nlm.nih.gov/RxNormAPIs.html#uLink=RxNorm_REST_getAllRelatedInfo
    """
    base_uri = 'http://rxnav.nlm.nih.gov/REST'
    url = '{base_uri}/rxcui/{rxcui}/allrelated'.format(base_uri = base_uri, rxcui = rxcui)
    response = requests.get(url)
    tree = xml.etree.ElementTree.fromstring(response.text)
    xml_ingredients = tree.findall("./allRelatedGroup/conceptGroup[tty='IN']/conceptProperties")
    for xml_ingredient in xml_ingredients:
        assert xml_ingredient.findtext('tty') == 'IN'
        yield tuple(xml_ingredient.findtext(tag) for tag in ['rxcui', 'name', 'umlscui'])

In [3]:
# Read the rxnorm mappings to ehrlink medications computed by antoine lizee
url = 'https://raw.githubusercontent.com/antoine-lizee/RRxNorm/master/Output/allMatchesWithProperties.csv'
med_df = pandas.read_csv(url, index_col=0)
med_df[:3]

,medication,medication_definition_id,indications,rxcui,score,rank,nAtoms,name,synonym,tty,language,suppress,umlscui
1,4-Aminopyridine Powder,108483,1,897018,67,5,12,dalfampridine,NaN,IN,ENG,N,C0000477
2,Abilify 1 MG/ML Oral Solution,123191,2,544412,100,1,7,aripiprazole 1 MG/ML Oral Solution [Abilify],Abilify 1 MG/ML Oral Solution,SBD,ENG,N,C1586223
3,Abilify 10 MG Oral Tablet,104197,11,352307,100,1,16,aripiprazole 10 MG Oral Tablet [Abilify],Abilify 10 MG Oral Tablet,SBD,ENG,N,C1169930


In [4]:
# Read the rxnorm mappings to ehrlink medications computed by antoine lizee

# Prepare the write file
path = os.path.join('data', 'rxnorm-as-ingredient.tsv')
write_file = open(path, 'w')
header = ['input_rxcui', 'input_tty', 'input_name', 'n_ingredients',
          'ingredient_rxcui', 'ingredient_name', 'ingredient_umlscui']
writer = csv.DictWriter(write_file, delimiter='\t', fieldnames=header)
writer.writeheader()

# Query the RxNorm API for each concept.
for rxcui, tty, name in set(zip(med_df.rxcui, med_df.tty, med_df.name)):
    ingredients = list(get_rxnorm_ingredients(rxcui))
    input_dict = {'input_rxcui': rxcui, 'input_tty': tty, 'input_name': name, 'n_ingredients': len(ingredients)}
    if not ingredients:
        print('No ingredients found for', rxcui, tty, name)
    for ingredient in ingredients:
        ingr_dict = input_dict.copy()
        for tag, value in zip(['ingredient_rxcui', 'ingredient_name', 'ingredient_umlscui'], ingredient):
            ingr_dict[tag] = value
        writer.writerow(ingr_dict)
    time.sleep(2)

# Close the completed file
write_file.close()

No ingredients found for 1151137 DFG Oral Liquid Product
No ingredients found for 7067 DF Mouthwash
